[Riddler solution](https://fivethirtyeight.com/features/can-you-solve-this-astronomical-enigma/)

In [15]:
#find the pairings that cross each other
N=11
constrain_crossings=[]
c=0
for a1 in range(N):
    for a2 in range(a1+2,a1+N-1):
        for b1 in range(a1+1,a2):
            for b2 in range(a2+1,a1+N):
                if a1%N<a2%N and b1%N<b2%N:
                    constrain_crossings.append((a1%N,a2%N,b1%N,b2%N))
                    c+=1

In [21]:
len(constrain_crossings)

330

In [31]:
#test
#find the pairings that cross each other
N=11
constrain_crossings_2=[]
c=0
for a1 in range(N):
    for a2 in range(a1+2,a1+9):
        for b1 in range(a1+1,a2):
            for b2 in range(a2+1,a1+N):
                if a1%N<a2%N and b1%N<b2%N:
                    constrain_crossings_2.append((a1%N,a2%N,b1%N,b2%N))
                    c+=1

In [32]:
len(constrain_crossings_2)

322

In [33]:
list(set(constrain_crossings)-set(constrain_crossings_2))

[(0, 9, 1, 10),
 (0, 9, 4, 10),
 (0, 9, 3, 10),
 (0, 9, 5, 10),
 (0, 9, 6, 10),
 (0, 9, 8, 10),
 (0, 9, 2, 10),
 (0, 9, 7, 10)]

In [9]:
from ortools.sat.python import cp_model

#LIP solution to a given arrangement
def findMax(sequence):
    #sequence=[1,4,8,7,11,2,5,9,3,6,10]
    model = cp_model.CpModel()
    N=len(sequence)
    pairings=[[0 for x in range(N)] for y in range(N)]
    for i in range(N):
        for j in range(i+1,N):
            #pairings[i][j]=1
            pairings[i][j]=model.NewIntVar(0,1,str(i)+","+str(j))
    #print(pairings)
    for s in constrain_crossings:
        model.Add(pairings[s[0]][s[1]]+pairings[s[2]][s[3]]<=1)

    for i in range(N):
        model.Add(sum(pairings[i][j]+pairings[j][i] for j in range(N))<=1)

    obj=0
    for i in range(N):
        for j in range(i+1,N):
            obj+=sequence[i]*sequence[j]*pairings[i][j]
    model.Maximize(obj)
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        total=0
        seq=""
        for i in range(N):
            for j in range(i+1,N):
                if solver.Value(pairings[i][j]):
                    #print(str(sequence[i])+","+str(sequence[j]))
                    total+=sequence[i]*sequence[j]
                    seq+="("+str(sequence[i])+","+str(sequence[j])+")"
    return total, seq

In [12]:
print(findMax([10,6,3,9,5,2,11,7,8,4,1]))

(237, '(10,6)(3,4)(9,11)(5,2)(7,8)')
